# Customer Segmentation Task

<img src= "https://thecxinsights.com/wp-content/uploads/2021/03/Customer-Segmentation-Featured-Image-3.png" style='width: 1200px;'>


In this notebook we are going to work on customer segmentation task.
Purpose of this work, helping authorised people of the Mall to increase the profits by analysing data given and extracting some insights from that. Customer segmentation has great importance in marketing to target ideal prospect clients. 

 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import pandas as pd 
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from scipy import stats
import matplotlib.gridspec as gridspec

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/customer-segmentation-tutorial-in-python/Mall_Customers.csv')
df.head()

In [ ]:
df.describe()

In the data, there are 200 samples.Data consists of 5 features as costumer id, gender, age, income and spending scores. To see what kind of distribution data has, lets visualize it. 

In [ ]:
df.rename(columns={'Annual Income (k$)':'Income','Spending Score (1-100)':'Spending_score'},inplace=True)


In [ ]:


gs = gridspec.GridSpec(1, 3)

fig = plt.figure(figsize=(18, 3))


fig.text(0.39, 1.09, 'Overall look into data ', 
         fontsize=18, fontweight='bold', fontfamily='serif')
fig.text(0.25, 1, 'Data looks close to Gaussian Distribution for Income and Spending Score features', 
         fontsize=12, fontweight='light', fontfamily='serif')


ax1 = fig.add_subplot(gs[0, 0]) 
ax1 = sns.distplot(df["Income"],
                   color='#244747',hist_kws=dict(edgecolor="white", linewidth=2,alpha=0.9))
ax1.set_facecolor("white")


ax2 = fig.add_subplot(gs[0, 1])
ax2 = sns.distplot(df["Spending_score"],
                   color='#244747',hist_kws=dict(edgecolor="white", linewidth=2,alpha=0.9))
ax2.set_facecolor("white")


ax3 = fig.add_subplot(gs[0, 2])
ax3 = sns.distplot(df['Age'],color='#244747',
                   hist_kws=dict(edgecolor="white", linewidth=2,alpha=0.9))
ax3.set_facecolor("white")



sns.despine()

plt.subplots_adjust(wspace=0.2)

print( 'Skewness of normal distribution for Age: {}'.format(stats.skew(df.Age)))

print( 'Skewness of normal distribution for Income : {}'.format(stats.skew(df.Income)))

print( 'Skewness of normal distribution for Spending score : {}'.format(stats.skew(df.Spending_score)))





Income and spending score seems like fairly close to normal distribution. However we can do a Skewness and Kurtosis test for Age feature as it seems like a bit skewed. If skewness is less than -1 or greater than 1, the distribution is highly skewed. Having looked at Skewness score, we can consider Age distribution is also close to normal distribution or at least it is not highly skewed. 


In [ ]:
values = df['Gender'].value_counts()
labels = ['Female','Male']

fig, ax = plt.subplots(figsize = (5,5), dpi = 100)
explode = (0.1,0)

patches, texts, autotexts = ax.pie(values, labels = labels, autopct = '%1.1f%%', shadow = True,
                                   startangle = 90, explode = explode,labeldistance=1.1,colors=['darkorange','skyblue'])


plt.show()

print('There are {} females and {} males'.format(values.Female,values.Male))

In [ ]:
df_=df.drop(['Gender','CustomerID'],axis=1) 
df_corr=df_.corr()

In [ ]:
heatmap = sns.heatmap(df_corr, vmin=-1, vmax=1, annot=True, cmap="Oranges_r")

There is no direct correlation between income and spending score although there is negatively a slight relationship between age and spending score

In [ ]:
males_income=df[df.Gender=='Male']['Income']
females_income=df[df.Gender=='Female']['Income']

my_bins = range(10,150,10)
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18,5))
sns.distplot(males_income, kde=False, color='#0066ff', ax=ax1, hist_kws=dict(edgecolor="k"))
ax1.set_xticks(my_bins)
ax1.set_yticks(range(0,24,2))
ax1.set_ylim(0,22)
ax1.set_title('Males')
ax1.set_ylabel('Count')
ax1.text(85,14, "Mean income: {:.1f}k$".format(males_income.mean()))
ax1.text(85,13, "Median income: {:.1f}k$".format(males_income.median()))
ax1.text(85,12, "Std. deviation: {:.1f}k$".format(males_income.std()))
ax1.set_facecolor("white")


sns.boxplot(x='Gender', y='Income', data=df, ax=ax2)
ax2.set_title('Annual Income')
ax2.text(0.1,100,'Kolgomorov-Smirnov test \n for two samples income\n p-value score  {:.2f}'.format(stats.ks_2samp(males_income, females_income)[1]))
ax2.set_facecolor("white")



sns.distplot(females_income, bins=my_bins, kde=False, color='#cc66ff', ax=ax3, hist_kws=dict(edgecolor="k"))
ax3.set_xticks(my_bins)
ax3.set_yticks(range(0,24,2))
ax3.set_ylim(0,22)
ax3.set_title('Females')
ax3.set_ylabel('Count')
ax3.text(85,14, "Mean income: {:.1f}k$".format(females_income.mean()))
ax3.text(85,13, "Median income: {:.1f}k$".format(females_income.median()))
ax3.text(85,12, "Std. deviation: {:.1f}k$".format(females_income.std()))

ax3.set_facecolor("white")

plt.show()

Between genders, as we can see there is 3k difference in mean of incomes with almost same std. However to be able to prove if there is statistical difference we do Kolgomorov-Smirnov Test. Kolmogorov-Smirnov test is used to check whether 2 samples follow the same distribution. By looking at the P-value score which is 0.78, we can confidentally say that Statistically there is no significant difference between female and male incomes. [For orginal graph](https://www.kaggle.com/datark1/customers-clustering-k-means-dbscan-and-ap)


In [ ]:
males_ss=df[df.Gender=='Male']['Spending_score']
females_ss=df[df.Gender=='Female']['Spending_score']

my_bins = range(0,105,5)
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18,5))
sns.distplot(males_ss, kde=False,bins=my_bins,color='#0066ff', ax=ax1, hist_kws=dict(edgecolor="k"))
ax1.set_xticks(my_bins)
ax1.set_yticks(range(0,24,2))
ax1.set_ylim(0,18)
ax1.set_title('Males')
ax1.set_ylabel('Count')
ax1.text(35,16, "Mean spending score: {:.1f}".format(males_ss.mean()))
ax1.text(35,15, "Median spending score: {:.1f}".format(males_ss.median()))
ax1.text(35,14, "Std. deviation: {:.1f}".format(males_ss.std()))
ax1.set_facecolor("white")

sns.boxplot(x='Gender', y='Spending_score', data=df, ax=ax2)
ax2.set_title('Spending Score')
ax2.text(0.1,75,'Kolgomorov-Smirnov test \n for two samples \nspending score \n p-value score  {:.2f}'.format(stats.ks_2samp(males_ss, females_ss)[1]))
ax2.set_facecolor("white")


sns.distplot(females_ss, bins=my_bins, kde=False, color='#cc66ff', ax=ax3, hist_kws=dict(edgecolor="k"))
ax3.set_xticks(my_bins)
ax3.set_yticks(range(0,24,2))
ax3.set_ylim(0,18)
ax3.set_title('Females')
ax3.set_ylabel('Count')
ax3.text(50,16, "Mean spending score: {:.1f}".format(females_ss.mean()))
ax3.text(50,15, "Median spending score: {:.1f}".format(females_ss.median()))
ax3.text(50,14, "Std. deviation: {:.1f}".format(females_ss.std()))

ax3.set_facecolor("white")


plt.show()


In terms of spending scores of both genders again statistically there is no significant difference according to P-value of Kolgomorov-Smirnov test which is 0.29 and smaller than 0.05 ( Null-Hypothesis )

For clustering, we are going to use K-means which is good unsupervised method especially for this amount of data. We will be doing clustring according to Spending score and Income relations but in this point we could have also tried Spending score- Age features too. 

In [ ]:
X = df[['Spending_score','Income']].iloc[: , :].values
inertia = []
for n in range(1 , 11):
    algorithm = (KMeans(n_clusters = n , n_init = 10 ,max_iter=500, 
                        tol=0.0001,  random_state= 82) )
    algorithm.fit(X)
    inertia.append(algorithm.inertia_)

In [ ]:
   
fig, ax = plt.subplots(figsize=(12, 8))
sns.lineplot(x=list(range(1, 11)), y=inertia, ax=ax)

ax.set_xlabel('Number of Clusters')
ax.set_ylabel('Inertia')

ax.set_title('Inertia Scores for Number of Clusters',size=15)

ax.annotate('Elbow point indicates number of clusters that will be choosen', size=13,color='black',xy=(5, 48000), xytext=(4, 150000), xycoords='data',          
             arrowprops=dict(arrowstyle='->', connectionstyle='arc3', color='red', lw=3))

#plt.style.use('seaborn')
ax.set_facecolor("white")
#plt.grid()


plt.show()

As we can see the graph above which shows inertia scores, we will go with 5 as number of clusters because we can see that is the elbow point 

In [ ]:
algorithm = (KMeans(n_clusters = 5 , n_init = 10 ,max_iter=500, 
                        tol=0.0001,algorithm='elkan',random_state= 82) )
algorithm.fit(X)
labels1 = algorithm.labels_

print(labels1)

In [ ]:
labels_=[]

for i in labels1:
    if i==0:
        labels_.append('low income-high spending score')
    elif i==1:
        labels_.append('high income-high spending score')
    elif i==2:
        labels_.append('high income-low spending score')
    elif i==3:
        labels_.append('mid income-mid spending score')
    elif i==4:
        labels_.append('low income-low spending score')
df['Labels']=labels_


age_group=[]

for i in df['Age']:
    if i >=18 and i <=24:
        age_group.append('18-24 years old')
    elif i >=25 and i<=35:
        age_group.append('25-35 years old')
    elif i >=36 and i <=45:
        age_group.append('36-45 years old')
    elif i >=46 and i<=55:
        age_group.append('46-55 years old')
    elif i >=56 and i<=70:
        age_group.append('56-70 years old')
        
df['Age_group']=age_group
df.head()

In [ ]:
plt.figure(1 , figsize = (15 , 7) )


plt.figure(figsize=(12, 8))
sns.scatterplot(df['Income'], df['Spending_score'], hue=df['Labels'],legend="full",s=150,
                palette=sns.color_palette("Set2", 5))
plt.ylabel('Spending Score') , plt.xlabel('Annual Income (k$)')
plt.show()

After clustering, we can see that there is a high income low spending group which is shown with light green. This might be our target with initial look into graph.

In [ ]:
fig = plt.figure(figsize=(20,8))

labels_1=df.Labels.unique()

ax = fig.add_subplot(121)
sns.swarmplot(x='Age', y='Labels', data=df, ax=ax,palette='Set1_r',size=10,edgecolor='black')
ax.set_yticklabels(labels_1,size=15)
ax.set_title('Clusters with Age')


plt.show()

**Low income high spending score** group has relatively narrow range of ages in which between 20’s and mid 30’s 
**Mid income mid spending score** group has  remarkably large range of age which is from 18’s to 70’s 
**High income high spending score** group's age is between mid 20s to early 40s 


In [ ]:
fig = plt.figure(figsize=(20,8))

labels_1=df.Labels.unique()

ax = fig.add_subplot(121)
sns.swarmplot(x='Spending_score', y='Labels', data=df, ax=ax,palette='Set1_r',size=10)
ax.set_yticklabels(labels_1,size=15)
ax.set_title('Clusters with Spending Scores')



plt.show()

**High income low spending group** has the worst spending score of the population.

In [ ]:
print('There are {} of customers who are in the group of mid income-mid spending score'.format(len(df[df.Labels=='mid income-mid spending score'])))

print('There are {} of customers who are in the group of high income-high spending score'.format(len(df[df.Labels=='high income-high spending score'])))

print('There are {} of customers who are in the group of high income-low spending score'.format(len(df[df.Labels=='high income-low spending score'])))



In [ ]:
values = [81,39,35,45]
labels = ['mid income-mid spending','high income-high spending','high income-low spending','Rest']

fig, ax = plt.subplots(figsize = (5,5), dpi = 100)


patches, texts, autotexts = ax.pie(values, labels = labels, autopct = '%1.1f%%', shadow = True,
                                   startangle = 140,labeldistance=1.1,colors=['darkorange','skyblue','lightgreen','gray'])
plt.show()



As it can be seen with Pie Chart above, Mid income mid spending score group is vast majority of the all customers in which has 81 people in. 

In [ ]:

import plotly.graph_objects as go

headerColor = 'skyblue'
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Groups</b>','<b>Age Mean</b>','<b>Income Mean</b>','<b>Spending Score Mean</b>'],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      ['Mid income-mid spending', 'High income-low spending', 'High income-high spending'],
      [42.71, 41.11, 39.69],
      [55.29, 88.20, 86.53],
      [ 49.51,17.11, 82.12]],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.show()

In [ ]:
df3=df[df.Labels=='high income-high spending score'].sort_values(by='Age',ascending=True)
#df3.describe()


In [ ]:
df2=df[df.Labels=='high income-low spending score'].sort_values(by='Age',ascending=True)
df2.describe()

In [ ]:

age=df2.groupby(['Gender'])['Age'].mean()
print('Age mean between genders: ',age)
print('Numbers: ',df2.Gender.value_counts())
print('Income mean for both genders',df2.groupby(['Gender'])['Income'].mean())
print('Spending scores for both genders:',df2.groupby(['Gender'])['Spending_score'].mean())


In [ ]:
fig = plt.figure(figsize=(20,8))

labels_2=df2.Age_group.unique()

df2.head(7)



In [ ]:
fig = plt.figure(figsize=(20,8))



ax = fig.add_subplot(121)
sns.countplot(y='Age_group',hue='Gender',data=df2,palette='Blues',edgecolor='white',lw=2.9)
ax.set_facecolor("white")
plt.grid() 
ax.set_title('Cluster of "High income-low spending score"')
plt.plot()

In [ ]:
fig = plt.figure(figsize=(20,12))



sns.catplot(
    data=df2, kind="bar",
    x="Age_group", y='Spending_score',col="Gender",
    ci="sd", palette='summer', alpha=.7, height=9,edgecolor='black'
)
sns.despine(left=True) 
#plt.gca().set_facecolor("white")



plt.plot()

Although females spending score looks like more balanced than males, both side should be taken into consideration. Because there are only 16 females and 19 males in the group which is not good enough to make strong conclusions between genders. 

# Conclusion

Between males and females, there is no considerable differences in terms of income and spending scores in this particular Mall Customer data with general view. Therefore it is not really necessary to focus one gender specifically. 
Two low income groups should not necessarily be taken into consideration as target groups because even though one of them has good spending score, they do not enough income to be considered.
Mid income group is the largest community of the Mall customers. Although they do not seem like primary target group, still they should be paid attention due to biggest part of the all clients. 
High income high spending group  is the prime people of the Mall as they have high incomes and highest spending scores which could be great for the source of profit. As they can be regular customers and happy with the facilities of the Mall, they should be kept satisfied. 
High income low spending group seems ideal target for the Mall as the group has highest income and potentially can spend more but interestingly they have the lowest spending score. They might be somewhat unhappy or unsatisfied with services of Mall. In the group, especially 3 sub-groups which are between 25-55 years olds should firstly be aimed to increase the spending